In [1]:
import os
import datasets
import numpy as np
import re
import nltk
from gensim.models.keyedvectors import KeyedVectors

nltk.download('stopwords')
from nltk.corpus import stopwords

c:\users\keshavpc\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keshavPC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset_path = os.path.join("data", "cornell")

In [3]:
data = datasets.readCornellData(dataset_path, max_len=100000)

100%|█████████████████████████████████████████████████████████████████████████| 83097/83097 [00:04<00:00, 20236.84it/s]


In [4]:
dataset_path = os.path.join("data", "opensubs/OpenSubtitles")

In [5]:
data += datasets.readOpensubsData(dataset_path, max_len=100000)

Loading OpenSubtitles conversations in data\opensubs/OpenSubtitles.


OpenSubtitles data files: 0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [10]:
data[-3:]

[('hello diamond ranch', 'no i m sorry she doesn t work here anymore'),
 ('i know', 'but if you come in we have a number of very beautiful girls'),
 ('but if you come in we have a number of very beautiful girls',
  'and i m sure you il find someone you can be satisfied with okay')]

In [29]:
len(data)

1431666

In [13]:
wv_embeddings = KeyedVectors.load_word2vec_format(
    'data/GoogleNews-vectors-negative300.bin',
    binary=True
)

In [15]:
# remove short sentences
data = [sentences for sentences in data if len(sentences[0].split()) > 2]

In [17]:
data[8]

('cesc ma tete this is my head', 'right see youre ready for the quiz')

In [18]:
len(data)

1638457

In [19]:
data[-2000]

('til they checked her over said there was nothing to be done',
 'took 2 months til she was stable enough to move')

In [20]:
data = [[sentences[0].split(), sentences[1].split()]for sentences in data]

In [21]:
data[0]

[['can',
  'we',
  'make',
  'this',
  'quick',
  'roxanne',
  'korrine',
  'and',
  'andrew',
  'barrett',
  'are',
  'having',
  'an',
  'incredibly',
  'horrendous',
  'public',
  'break',
  'up',
  'on',
  'the',
  'quad',
  'again'],
 ['well',
  'i',
  'thought',
  'wed',
  'start',
  'with',
  'pronunciation',
  'if',
  'thats',
  'okay',
  'with',
  'you']]

In [22]:
data = [sentences for sentences in data if len(sentences[0]) >= 3]
data = [sentences for sentences in data if len(sentences[0]) <= 20]
data = [sentences for sentences in data if len(sentences[1]) >= 3]
data = [sentences for sentences in data if len(sentences[1]) <= 20]

In [24]:
from collections import defaultdict
word_count = defaultdict(int)
for sentences in data:
    for word in sentences[0]:
        word_count[word] += 1
    for word in sentences[1]:
        word_count[word] += 1

In [25]:
wc_threshold = 50

In [26]:
# remove the lines with unknown words
data_filtered = []
for sentence1, sentence2 in data:
    valid = True
    for word in sentence1 + sentence2:
        if word not in wv_embeddings or word_count[word] < wc_threshold:
            valid = False
            break
    if valid:
        data_filtered.append([sentence1, sentence2])

In [30]:
len(data_filtered)

379897

In [31]:
data = data_filtered

In [32]:
data.sort(key=lambda x:len(x[0]))

In [36]:
data[10]

[['is', 'that', 'right'],
 ['yeah', 'but', 'then', 'i', 'fucked', 'up', 'i', 'fell', 'for', 'her']]

In [37]:
len(data)

379897

In [38]:
word_set = set()
for sentence1, sentence2 in data:
    word_set |= set(sentence1)
    word_set |= set(sentence2)

In [39]:
len(word_set)

19926

In [40]:
start_token = "[START]"
end_token = "[END]"
pad_token = "[PAD]"

In [41]:
idx_to_word = [start_token, end_token, pad_token] + [None] * len(word_set)
word_to_idx = {start_token:0, end_token:1, pad_token:2}
index = 3
for word in word_set:
    word_to_idx[word] = index
    idx_to_word[index] = word
    index += 1

In [61]:
start_idx, end_idx = 0, 1

In [51]:
embedding_dim = 300
word_embeddings = np.zeros((len(word_set)+3, embedding_dim), dtype='float32')
word_embeddings[0,:] = 0.
word_embeddings[1,:] = 1.
word_embeddings[2,:] = -1.
for i in range(3, len(word_set)+3):
    word_embeddings[i, :] = wv_embeddings[idx_to_word[i]]   

In [43]:
for sentences in data:
    sentences[1].append(end_token)

In [44]:
data[10]

[['is', 'that', 'right'],
 ['yeah',
  'but',
  'then',
  'i',
  'fucked',
  'up',
  'i',
  'fell',
  'for',
  'her',
  '[END]']]

In [45]:
input_lengths = [len(sentences[0]) for sentences in data]
ground_truth_lengths = [len(sentences[1]) for sentences in data]

In [46]:
# add padding
max_input_lengths = max(input_lengths)
max_ground_truth_lengths = max(ground_truth_lengths)
input_sentences = []
ground_truth_sentences = []
for sentences in data:
    input_sentences.append(sentences[0] + [pad_token]*(max_input_lengths-len(sentences[0])))
    ground_truth_sentences.append(sentences[1] + [pad_token]*(max_ground_truth_lengths-len(sentences[1])))

In [47]:
input_sentences_idx = [[word_to_idx[word] if word in word_to_idx else unknown_idx for word in sentence] for sentence in input_sentences]
ground_truth_sentences_idx = [[word_to_idx[word] if word in word_to_idx else unknown_idx for word in sentence] for sentence in ground_truth_sentences]

In [48]:
input_sentences_idx = np.array(input_sentences_idx)

In [49]:
ground_truth_sentences_idx = np.array(ground_truth_sentences_idx)
input_lengths = np.array(input_lengths)
ground_truth_lengths = np.array(ground_truth_lengths)

In [50]:
data[0]

[['you', 'know', 'chastity'],
 ['i', 'believe', 'we', 'share', 'an', 'art', 'instructor', '[END]']]

In [52]:
def batch_generator(batch_size,
                    input_sentences_idx,
                    ground_truth_sentences_idx,
                    input_lengths,
                    ground_truth_lengths):
    index = 0
    while index < len(input_sentences_idx):
        batch_input_length = input_lengths[index:index+batch_size]
        batch_input_sentences_idx = input_sentences_idx[index:index+batch_size, :batch_input_length.max()]
        batch_ground_truth_length = ground_truth_lengths[index:index+batch_size]
        batch_ground_truth_sentences_idx = ground_truth_sentences_idx[index:index+batch_size, :batch_ground_truth_length.max()]
        yield (batch_input_sentences_idx, batch_input_length,
              batch_ground_truth_sentences_idx, batch_ground_truth_length)
        index += batch_size

In [53]:
g = batch_generator(10, input_sentences_idx, ground_truth_sentences_idx, input_lengths, ground_truth_lengths)

In [54]:
vocab_size = len(word_set) + 3
num_units = 128
embedding_size = 300
num_encoder_layers = 2
num_decoder_layers = 2

In [55]:
import tensorflow as tf
from tensorflow.contrib import layers

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [56]:
def text_prepare(text):
    """Performs tokenization and simple preprocessing."""
    
    replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
    good_symbols_re = re.compile('[^0-9a-z #+_]')
#     stopwords_set = set(stopwords.words('english'))

    text = text.lower()
    text = replace_by_space_re.sub(' ', text)
    text = good_symbols_re.sub('', text)
#     text = ' '.join([x for x in text.split() if x and x not in stopwords_set])

    return text.strip()

In [58]:
class ChatBot:
    def __init__(self):
        self.declare_placeholders()
        self.build_input_encoder()
        self.build_ground_truth_encoder()
        self.build_hidden_state()
        self.build_decoder()
        self.define_loss_and_train()
    
    def declare_placeholders(self):
        """Specifies placeholders for the model."""
        # Placeholders for input and its actual lengths.
        self.input_batch = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input_batch')
        self.input_batch_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_batch_lengths')

        # Placeholders for groundtruth and its actual lengths.
        self.ground_truth = tf.placeholder(shape=(None, None), dtype=tf.int32, name='ground_truth')
        self.ground_truth_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='ground_truth_lengths')

        self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
        self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[])

    def build_input_encoder(self):
        with tf.variable_scope('input_encoder') as input_encoder_scope:
#             random_initializer = tf.random_uniform((vocab_size, embedding_size), -1.0, 1.0)
            self.embeddings = tf.Variable(initial_value=word_embeddings, name='embeddings', dtype=tf.float32) 

            # Perform embeddings lookup for self.input_batch. 
            self.input_batch_embedded = tf.nn.embedding_lookup(self.embeddings, self.input_batch)
            # Create encoder cells
            rnn_layers = []
            for i in range(num_encoder_layers-1):
                with tf.variable_scope('input_encoder_rnn_layer' + str(i + 1)) as scope:
                    cell = tf.nn.rnn_cell.GRUCell(num_units, activation=tf.nn.relu)
                    cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                    rnn_layers.append(cell)
            with tf.variable_scope('input_encoder_rnn_layer' + str(num_encoder_layers)) as scope:
                cell = tf.nn.rnn_cell.GRUCell(num_units)
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                rnn_layers.append(cell)
            encoder_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
            self.input_encoder_outputs, self.final_input_encoder_state = tf.nn.dynamic_rnn(
                encoder_cell,
                self.input_batch_embedded,
                sequence_length=self.input_batch_lengths,
                dtype=tf.float32
            )
            self.final_input_encoder_state = self.final_input_encoder_state[-1]

    def build_ground_truth_encoder(self):
        with tf.variable_scope('ground_truth_encoder') as ground_truth_encoder:
            self.ground_truth_batch_embedded = tf.nn.embedding_lookup(self.embeddings, self.ground_truth)
            rnn_layers = []
            for i in range(num_encoder_layers-1):
                with tf.variable_scope('ground_truth_encoder_rnn_layer' + str(i + 1)) as scope:
                    cell = tf.nn.rnn_cell.GRUCell(num_units, activation=tf.nn.relu)
                    cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                    rnn_layers.append(cell)
            with tf.variable_scope('ground_truth_encoder_rnn_layer' + str(num_encoder_layers)) as scope:
                cell = tf.nn.rnn_cell.GRUCell(num_units)
                cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=self.dropout_ph, dtype=tf.float32)
                rnn_layers.append(cell)
            encoder_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
            _, self.final_ground_truth_encoder_state = tf.nn.dynamic_rnn(
                encoder_cell,
                self.ground_truth_batch_embedded,
                sequence_length=self.ground_truth_lengths,
                dtype=tf.float32
            )
            self.final_ground_truth_encoder_state = self.final_ground_truth_encoder_state[-1]

    def build_hidden_state(self):
        self.z_mean_from_input, self.z_log_var_from_input = tf.split(
            self.final_input_encoder_state, num_or_size_splits=2, axis=1)
        
        self.z_mean_from_ground_truth, self.z_log_var_from_grount_truth = tf.split(
            self.final_input_encoder_state, num_or_size_splits=2, axis=1)
        
        
        self.z = (self.z_mean_from_ground_truth +
                  tf.exp(0.5*self.z_log_var_from_grount_truth) *
                  tf.random_normal(tf.shape(self.z_log_var_from_grount_truth), 0, 1, dtype=tf.float32))

    def build_decoder(self):
        batch_size = tf.shape(self.input_batch)[0]
        start_tokens = tf.fill([batch_size], start_idx)
        ground_truth_as_input = tf.concat([tf.expand_dims(start_tokens, 1), self.ground_truth], 1)
        self.ground_truth_embedded = tf.nn.embedding_lookup(
            self.embeddings, ground_truth_as_input)
        train_helper = tf.contrib.seq2seq.TrainingHelper(self.ground_truth_embedded,
                                                         self.ground_truth_lengths)
        infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(self.embeddings, start_tokens, end_idx)

        def decode(helper, scope, reuse=None):
            """Creates decoder and return the results of the decoding with a given helper."""
            with tf.variable_scope(scope, reuse=reuse):
                # Create GRUCell with dropout. Do not forget to set the reuse flag properly.
                rnn_layers = []
                for i in range(num_decoder_layers-1):
                    with tf.variable_scope('decoder_rnn_layer' + str(i + 1)) as scope:
                        decoder_cell = tf.contrib.rnn.GRUCell(num_units=num_units/2, reuse=reuse, activation=tf.nn.tanh)
                        decoder_cell = tf.contrib.rnn.DropoutWrapper(decoder_cell, input_keep_prob=self.dropout_ph)
                        rnn_layers.append(decoder_cell)
                with tf.variable_scope('decoder_rnn_layer' + str(num_decoder_layers)) as scope:
                    decoder_cell = tf.contrib.rnn.GRUCell(num_units=num_units/2, reuse=reuse)
                    decoder_cell = tf.contrib.rnn.DropoutWrapper(decoder_cell, input_keep_prob=self.dropout_ph)
                    rnn_layers.append(decoder_cell)
                decoder_cell = tf.contrib.rnn.MultiRNNCell(rnn_layers)
                # Create attention
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    num_units=num_units, memory=tf.split(self.input_encoder_outputs, num_or_size_splits=2, axis=-1)[0],
                    memory_sequence_length=self.input_batch_lengths)
                decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
                    decoder_cell, attention_mechanism, attention_layer_size=num_units)
                # Create a projection wrapper.
                decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(decoder_cell, vocab_size, reuse=reuse)
                # Create BasicDecoder, pass the defined cell, a helper, and initial state.
                # The initial state should be equal to the final state of the encoder!
                second_state = tf.zeros((1, num_units/2))
                second_state = tf.tile(second_state, [batch_size, 1])
                decoder_initial_state = decoder_cell.zero_state(dtype=tf.float32, batch_size=batch_size).clone(
                    cell_state=(self.z, second_state))
                decoder = tf.contrib.seq2seq.BasicDecoder(cell=decoder_cell, helper=helper, initial_state=decoder_initial_state)

                # The first returning argument of dynamic_decode contains two fields:
                #   rnn_output (predicted logits)
                #   sample_id (predictions)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, maximum_iterations=tf.reduce_max(self.ground_truth_lengths), 
                                                                  output_time_major=False, impute_finished=True)

                return outputs

        self.train_outputs = decode(train_helper, 'decode')
        self.infer_outputs = decode(infer_helper, 'decode', reuse=True)
        self.train_predictions = self.train_outputs.sample_id
        self.infer_predictions = self.infer_outputs.sample_id

    def define_loss_and_train(self):
        weights = tf.cast(tf.sequence_mask(self.ground_truth_lengths), dtype=tf.float32)
        self.reconstruction_loss = tf.contrib.seq2seq.sequence_loss(
            self.train_outputs.rnn_output,
            self.ground_truth,
            weights
        )
        self.kl_loss = (0.5*(self.z_log_var_from_input-self.z_log_var_from_grount_truth)+
                       (tf.exp(self.z_log_var_from_grount_truth)+(self.z_mean_from_ground_truth-self.z_mean_from_input)**2)/
                       (2*tf.exp(self.z_log_var_from_input))-0.5)
        self.kl_loss = tf.reduce_mean(self.kl_loss)
        self.loss = self.kl_loss + self.reconstruction_loss
        self.train_op = tf.contrib.layers.optimize_loss(
            loss=self.loss,
            optimizer='Adam',
            learning_rate=self.learning_rate_ph,
            clip_gradients=1.0,
            global_step=tf.train.get_global_step()
        )

    def train_on_batch(self, session, X, X_seq_len, Y, Y_seq_len, learning_rate, dropout_keep_probability):
        feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: X_seq_len,
            self.ground_truth: Y,
            self.ground_truth_lengths: Y_seq_len,
            self.learning_rate_ph: learning_rate,
            self.dropout_ph: dropout_keep_probability
        }
        loss, _ = session.run([
            self.loss,
            self.train_op], feed_dict=feed_dict)
        return loss
    
    def get_reply(self, session, input_sentence):
        input_sentence = text_prepare(input_sentence)
        X = [[word_to_idx[word] if word in word_to_idx else unknown_idx for word in input_sentence]]
        X = np.array(X)
        feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: np.array([len(input_sentence)])
        }
        z_mean, z_log_var, input_encoder_outputs = session.run([
            self.z_mean_from_input,
            self.z_log_var_from_input,
            self.input_encoder_outputs
        ], feed_dict=feed_dict)
        z_mean = z_mean[0]
        z_log_var = z_log_var[0]
        z = np.random.normal(z_mean, np.exp(0.5*z_log_var), z_mean.size)
        z = z[np.newaxis,:]
        feed_dict = {
            self.z: z,
            self.input_batch: X,
            self.input_batch_lengths: np.array([len(input_sentence)]),
            self.ground_truth_lengths: np.array([15])
        }
        pred = session.run([self.infer_predictions], feed_dict=feed_dict)
        return " ".join([idx_to_word[index] for index in pred[0][0]])

    def train(self, session, epochs, batch_size, input_sentences_idx, ground_truth_sentences_idx, input_lengths, ground_truth_lengths, learning_rate, dropout_keep_probability):
        for i in range(epochs):
            batch_num = 1
            for (batch_input_sentences_idx,
                 batch_input_length,
                 batch_ground_truth_sentences_idx,
                 batch_ground_truth_length) in batch_generator(
                batch_size, input_sentences_idx, ground_truth_sentences_idx,
                input_lengths, ground_truth_lengths):
                loss = self.train_on_batch(
                    session,
                    batch_input_sentences_idx,
                    batch_input_length,
                    batch_ground_truth_sentences_idx,
                    batch_ground_truth_length,
                    learning_rate,
                    dropout_keep_probability
                )
                print("Epoch {i}, batch {batch}, loss = {loss}".format(i=i+1, batch=batch_num, loss=loss))
                batch_num += 1

In [62]:
tf.reset_default_graph()

In [63]:
chatbot = ChatBot()

In [64]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
chatbot.train(session,
              1, 256,
              input_sentences_idx,
              ground_truth_sentences_idx,
              input_lengths,
              ground_truth_lengths,
              1e-4,
              0.5)

Epoch 1, batch 1, loss = 9.899627685546875
Epoch 1, batch 2, loss = 9.899077415466309
Epoch 1, batch 3, loss = 9.898760795593262
Epoch 1, batch 4, loss = 9.898591041564941
Epoch 1, batch 5, loss = 9.897552490234375
Epoch 1, batch 6, loss = 9.89767074584961
Epoch 1, batch 7, loss = 9.896485328674316
Epoch 1, batch 8, loss = 9.896812438964844
Epoch 1, batch 9, loss = 9.895785331726074
Epoch 1, batch 10, loss = 9.895318984985352
Epoch 1, batch 11, loss = 9.895915985107422
Epoch 1, batch 12, loss = 9.894783020019531
Epoch 1, batch 13, loss = 9.892687797546387
Epoch 1, batch 14, loss = 9.894875526428223
Epoch 1, batch 15, loss = 9.892996788024902
Epoch 1, batch 16, loss = 9.891974449157715
Epoch 1, batch 17, loss = 9.891645431518555
Epoch 1, batch 18, loss = 9.890048027038574
Epoch 1, batch 19, loss = 9.888581275939941
Epoch 1, batch 20, loss = 9.88957405090332
Epoch 1, batch 21, loss = 9.887972831726074
Epoch 1, batch 22, loss = 9.889283180236816
Epoch 1, batch 23, loss = 9.888226509094238

Epoch 1, batch 186, loss = 5.736809253692627
Epoch 1, batch 187, loss = 5.926025390625
Epoch 1, batch 188, loss = 5.9170708656311035
Epoch 1, batch 189, loss = 5.84435510635376
Epoch 1, batch 190, loss = 5.814812660217285
Epoch 1, batch 191, loss = 5.97196102142334
Epoch 1, batch 192, loss = 6.279820919036865
Epoch 1, batch 193, loss = 6.136571884155273
Epoch 1, batch 194, loss = 6.129721641540527
Epoch 1, batch 195, loss = 6.123713493347168
Epoch 1, batch 196, loss = 6.160660266876221
Epoch 1, batch 197, loss = 6.067513465881348
Epoch 1, batch 198, loss = 6.307403564453125
Epoch 1, batch 199, loss = 6.037540435791016
Epoch 1, batch 200, loss = 5.996161460876465
Epoch 1, batch 201, loss = 6.182076930999756
Epoch 1, batch 202, loss = 6.04791784286499
Epoch 1, batch 203, loss = 6.016354560852051
Epoch 1, batch 204, loss = 6.001806259155273
Epoch 1, batch 205, loss = 5.908658027648926
Epoch 1, batch 206, loss = 5.962727069854736
Epoch 1, batch 207, loss = 5.919088363647461
Epoch 1, batch 

Epoch 1, batch 369, loss = 5.718180179595947
Epoch 1, batch 370, loss = 5.601260662078857
Epoch 1, batch 371, loss = 5.7197041511535645
Epoch 1, batch 372, loss = 5.823256015777588
Epoch 1, batch 373, loss = 5.7688889503479
Epoch 1, batch 374, loss = 5.759445667266846
Epoch 1, batch 375, loss = 5.760220050811768
Epoch 1, batch 376, loss = 5.808256149291992
Epoch 1, batch 377, loss = 5.731132507324219
Epoch 1, batch 378, loss = 5.822715759277344
Epoch 1, batch 379, loss = 5.749683856964111
Epoch 1, batch 380, loss = 5.616331577301025
Epoch 1, batch 381, loss = 5.647697448730469
Epoch 1, batch 382, loss = 5.710161209106445
Epoch 1, batch 383, loss = 5.593897819519043
Epoch 1, batch 384, loss = 5.785258769989014
Epoch 1, batch 385, loss = 5.614696502685547
Epoch 1, batch 386, loss = 5.685454368591309
Epoch 1, batch 387, loss = 5.804566860198975
Epoch 1, batch 388, loss = 5.8603835105896
Epoch 1, batch 389, loss = 5.874810218811035
Epoch 1, batch 390, loss = 5.625037670135498
Epoch 1, batc

Epoch 1, batch 551, loss = 5.782552719116211
Epoch 1, batch 552, loss = 5.710867881774902
Epoch 1, batch 553, loss = 5.805628299713135
Epoch 1, batch 554, loss = 5.732217788696289
Epoch 1, batch 555, loss = 5.748960494995117
Epoch 1, batch 556, loss = 5.734989643096924
Epoch 1, batch 557, loss = 5.839034080505371
Epoch 1, batch 558, loss = 5.5739359855651855
Epoch 1, batch 559, loss = 5.670580863952637
Epoch 1, batch 560, loss = 5.726480007171631
Epoch 1, batch 561, loss = 5.644850730895996
Epoch 1, batch 562, loss = 5.847506046295166
Epoch 1, batch 563, loss = 5.873259544372559
Epoch 1, batch 564, loss = 5.7432780265808105
Epoch 1, batch 565, loss = 5.669466495513916
Epoch 1, batch 566, loss = 5.665438175201416
Epoch 1, batch 567, loss = 5.68215274810791
Epoch 1, batch 568, loss = 5.768172740936279
Epoch 1, batch 569, loss = 5.722362995147705
Epoch 1, batch 570, loss = 5.872249603271484
Epoch 1, batch 571, loss = 5.720802307128906
Epoch 1, batch 572, loss = 5.910502910614014
Epoch 1, 

Epoch 1, batch 733, loss = 5.59346866607666
Epoch 1, batch 734, loss = 5.50395393371582
Epoch 1, batch 735, loss = 5.612408638000488
Epoch 1, batch 736, loss = 5.614148139953613
Epoch 1, batch 737, loss = 5.9874725341796875
Epoch 1, batch 738, loss = 5.7712626457214355
Epoch 1, batch 739, loss = 5.830273151397705
Epoch 1, batch 740, loss = 5.873953819274902
Epoch 1, batch 741, loss = 5.836766719818115
Epoch 1, batch 742, loss = 5.695374965667725
Epoch 1, batch 743, loss = 5.83892822265625
Epoch 1, batch 744, loss = 5.807259559631348
Epoch 1, batch 745, loss = 5.911267280578613
Epoch 1, batch 746, loss = 5.885354518890381
Epoch 1, batch 747, loss = 5.861804485321045
Epoch 1, batch 748, loss = 5.703033447265625
Epoch 1, batch 749, loss = 5.851438045501709
Epoch 1, batch 750, loss = 5.753206729888916
Epoch 1, batch 751, loss = 5.743318557739258
Epoch 1, batch 752, loss = 5.750457763671875
Epoch 1, batch 753, loss = 5.745096683502197
Epoch 1, batch 754, loss = 5.816944599151611
Epoch 1, ba

In [153]:
chatbot.get_reply(session, "who are you")

'i [END]'

In [2]:
import tensorflow as tf

In [5]:
x = tf.random_normal(tf.shape(100), 0, 1, dtype=tf.float32)

In [11]:
x.eval()

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [12]:
sess = tf.Session()

In [18]:
with sess.as_default():
    a = (x.eval())
    

In [19]:
a

1.1680882